In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
import pandas as pd
import scipy.optimize as opt

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [9]:
moments = np.array([-2, -1, 2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200, 6400])
realizations = 10

multifractal_data = np.load("../../data/multifractal_data.npy")

array([[[1.25528489, 1.12266341, 0.84221382, 0.79524715, 0.76292436],
        [1.14409683, 1.07237182, 0.88625576, 0.84497852, 0.81567478],
        [1.09682508, 1.04881406, 0.91448689, 0.88315214, 0.8604748 ],
        [1.07226228, 1.03730952, 0.9337137 , 0.90956724, 0.88736149],
        [1.05571566, 1.02937335, 0.94737715, 0.92751904, 0.90975241],
        [1.04610793, 1.02389464, 0.95679383, 0.94110469, 0.92585149],
        [1.03754322, 1.01978488, 0.96461653, 0.9512328 , 0.93925504]],

       [[1.27278053, 1.13133456, 0.82737167, 0.78046897, 0.74512069],
        [1.16177213, 1.08259992, 0.86681572, 0.82452122, 0.79649323],
        [1.11768623, 1.06067014, 0.89555819, 0.86101031, 0.83463272],
        [1.09364989, 1.04859078, 0.9138916 , 0.88484099, 0.86296645],
        [1.07906854, 1.04107334, 0.92791357, 0.90235475, 0.882217  ],
        [1.066867  , 1.03562815, 0.93818036, 0.91560672, 0.89774597],
        [1.05924612, 1.03146725, 0.94559621, 0.92601708, 0.90939342]],

       [[1.29525

In [5]:
# this is an (Nx5) matrix where each row has the following structure
# row 1: IPR_avg_1 | IPR_err_1 | L_1 | p_1 | lamb_1
# row 2: IPR_avg_2 | IPR_err_2 | L_2 | p_2 | lamb_2
# and so on...

ipr_data = np.load("../../data/AVG_IPR_joined_data.npy")
ipr_df = pd.DataFrame(ipr_data, columns = ['average', 'error', 'L', 'p', 'lambda'])
ipr_ps = np.sort(pd.unique(ipr_df['p']))
ipr_Ls = np.array(np.sort(pd.unique(ipr_df['L'])), dtype = int)


In [6]:
def f(x, a, c2): return c2*(x**a)

powers = []
gradients = []
for p in ipr_ps:
    curve_data = ipr_df[ipr_df['p'] == p]
    res = opt.curve_fit(
        f, curve_data['L'], 1/curve_data['average'], 
        bounds=([0, -np.inf], [np.inf, np.inf])
    )
    powers.append(res[0][0])
    gradients.append(res[0][1])

In [11]:
%matplotlib qt

plot_moment = [0, 1, 1, 1, 0]
plot_p = [0, 0, 1, 1, 1, 1, 0]

# Plotting parameters
scaling = 1.0
title_font_size = 10*scaling
plt.rcParams['xtick.major.pad'] = '-1.5'
plt.rcParams['ytick.major.pad'] = '-1.5'
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    #"text.usetex": True,
})

# Figure setup
pt = 0.0138889
fig, ax = plt.subplots(
    nrows=4, figsize=(246*pt*scaling, 250*pt*scaling),
    gridspec_kw={
        'left': 0.13, 'right': 0.99, 'top': 0.985, 'bottom': 0.065, 'hspace': 0.4,
        'height_ratios': (0.3, 0.15, 0.15, 0.15)
    }
)

# IPR data
for L in ipr_Ls:
    if L != 1000:
        ax[0].plot(
            ipr_ps, 
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            ls = '--'
        )
for L in ipr_Ls:
    if L != 1000:
        ax[0].scatter(
            ipr_ps,
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            marker='o',
            label=r'{}'.format(L),
            s=5
        )
ax[0].set_xlabel(r"$p$", labelpad=-10*scaling)
ax[0].set_ylabel(r"$\mathrm{PR}$", labelpad=-1)
ax[0].set_xticks((0, 2, 4, 6, 8, 10))
ax[0].tick_params(axis='x', pad=2)
ax[0].set_yscale('log')
#ax[0].legend(
#    loc=7, 
#    bbox_to_anchor=(0.0, 0.0, 1.22, 1.0), 
#    borderpad=0.1, 
#    handletextpad=0.0,
#    handlelength=1.0
#)

# IPR inset
inset_ax = inset_axes(ax[0], width="50%", height="55%", loc=4,
                   bbox_to_anchor=(.0, .32, 0.95, 0.98), bbox_transform=ax[0].transAxes)
inset_ax.set_xlabel(r"$p$", labelpad=-9*scaling)
#inset_ax.set_title(r"$\mathrm{PR}(p)=p^a/c$", fontsize=title_font_size, pad=-10)
inset_ax.set_xticks((0, 3, 7, 10))

color = (0.05, 0.4, 0.6)
inset_ax.plot(ipr_ps, np.abs(powers), color=color)
inset_ax.set_ylabel(r"$a$", color=color, labelpad=-9*scaling)
inset_ax.tick_params(axis='y', labelcolor=color)
inset_ax.tick_params(axis='both', pad=2)

color = (1.0, 0.5, 0.2)
inset_ax_twin = inset_ax.twinx()
inset_ax_twin.plot(ipr_ps, 1/np.abs(gradients), color=color)
inset_ax_twin.set_ylabel(r"$c$", color=color, labelpad=-9*scaling)
inset_ax_twin.tick_params(axis='y', labelcolor=color)
inset_ax_twin.set_yticks((0.0, 2.0))
inset_ax_twin.tick_params(axis='y', pad=2)

y_labels = [r"$D_{-2}$", r"$D_{-1}$", r"$D_{2}$", r"$D_{3}$", r"$D_{4}$"]
# Multifractality data
for (k, q) in enumerate(moments):
    if plot_moment[k]:
        for (i, p) in enumerate(ps):
            if plot_p[i]:
                ax[k].scatter(
                    system_sizes, 
                    multifractal_data[i, :, k],
                    linewidth=2.0, label="{}".format(p),
                    s=5
                )
                ax[k].plot(
                    system_sizes, 
                    multifractal_data[i, :, k], 
                    linewidth=0.5,
                    ls='--'
                )
        ax[k].set_xlabel(r"$L$", labelpad=-9*scaling)
        ax[k].set_ylabel(y_labels[k], labelpad=-1)
        ax[k].set_xscale('log')
        #ax[k].set_title(r"$q={}$".format(q), fontsize=title_font_size, pad=-10)
        ax[k].tick_params(axis='y', pad=1)
#ax[2].legend(
#    loc=7, 
#    bbox_to_anchor=(0.0, 0.0, 1.215, 1.0),
#    borderpad=0.1,
#    handletextpad=0.0,
#    handlelength=1.0
#)
plt.show()


-1
0.95
[1.14435653 1.09840557 1.07569206 1.06597213 1.05915579 1.05397446
 1.05087815]
-1
1.0
[1.1559442  1.11229091 1.0982019  1.08788948 1.08488993 1.08251568
 1.08123204]
-1
1.05
[1.17697325 1.13603174 1.12109705 1.11749116 1.11992514 1.12552601
 1.13079846]
-1
1.1
[1.19592578 1.1590008  1.15657179 1.16265549 1.17126575 1.1879949
 1.20642764]
2
0.95
[0.80787772 0.85094277 0.87484072 0.89198718 0.90272494 0.91019251
 0.91669719]
2
1.0
[0.79444604 0.83197272 0.85004053 0.86120932 0.86767527 0.87105102
 0.87238513]
2
1.05
[0.76287927 0.8018449  0.81520527 0.81963043 0.82104089 0.81694364
 0.80962283]
2
1.1
[0.74857861 0.77270996 0.77998677 0.77539298 0.76325195 0.74796396
 0.72772359]
3
0.95
[0.75305778 0.80695633 0.83731116 0.85723389 0.86878069 0.88005552
 0.88826743]
3
1.0
[0.74080528 0.78120311 0.80320346 0.82021533 0.82824282 0.83387038
 0.835459  ]
3
1.05
[0.71700624 0.7509545  0.76742561 0.7729666  0.77426038 0.77098711
 0.76293762]
3
1.1
[0.67738203 0.71825016 0.72902468 0.725

In [8]:
plt.savefig("../../paper_figs/fractal_dimensions.eps")
